In [2]:
import torch.nn as nn
import sqlalchemy
import pandas as pd
import torch as T
from torch.utils.data import Dataset
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import normalize, StandardScaler


class SqliteDataSet(Dataset):
    def __init__(self,
                 sqlite_rul='sqlite:///D:\\sqlite\\data\\finance.db',
                 query='select * from STOCK_DAILY',
                 transform=None):

        self.engine = sqlalchemy.create_engine('sqlite:///D:\\sqlite\\data\\finance.db')
        self.transform = transform
        self.result_set = pd.read_sql_query(query,self.engine.connect()).dropna()  
        self.result_set.reset_index(drop=True,inplace=True),
        X_columns=['Open','High' ,'Low' ,'Close','Adj Close' ,'Volume' ,'MACD_12_26_9' ,'MACDh_12_26_9' ,'MACDs_12_26_9','RSI_14' ,'BBL_5_2.0' ,'BBM_5_2.0' ,'BBU_5_2.0' ,'BBB_5_2.0' ,'BBP_5_2.0' ,'OBV' ,'SMA_20' ,'EMA_50' ,'STOCHk_14_3_3' ,'STOCHd_14_3_3' ,'ADX_14' ,'DMP_14' ,'DMN_14' ,'AD' ,'STDEV_30' ,'VWAP_D' ,'PV_DERIVATIVE' ]
        self.result_set['rate'] = ( self.result_set['Y_CLOSE5_MIN'] - self.result_set['Close'] ) / self.result_set['Close'] * 100
        sd_scaler = StandardScaler()
        self.result_set[X_columns] = sd_scaler.fit_transform(self.result_set[X_columns])

        self.length = self.result_set.shape[0]
        # from sklearn_pandas import DataFrameMapper

        # mapper = DataFrameMapper([(df.columns, StandardScaler())])
        # scaled_features = mapper.fit_transform(df.copy(), 4)
        # scaled_features_df = pd.DataFrame(scaled_features, index=df.index, columns=df.columns)

    def __getitem__(self, index) -> T.Tuple[T.Tensor,int]:
        data = self.result_set.loc[index]
        # rate = ( data['Y_CLOSE5_MAX'] - data['Close'] ) / data['Close'] * 100
        rate = data['rate']
        x = T.tensor(data.drop(['Y_CLOSE5_MIN','rate'])).to(T.float32)        
        if rate > 5:
            y = 4
        elif 2 < rate < 5:
            y = 3
        elif rate < -5:
            y = 0
        elif -5 <= rate < -2:
            y = 1
        else:
            y = 2         
       
        if self.transform:
            x = self.transform(x)
        return x, y

    def __len__(self):
        return self.length




import matplotlib.pyplot as plt

# Device configuration
device = T.device('cuda' if T.cuda.is_available() else 'cpu')
print(device)
# Hyper-parameters
input_size = 27
hidden_size = 200
num_classes = 5
num_epochs = 20
batch_size = 3
learning_rate = 0.01

query =r"""select Open ,High ,Low ,Close ,"Adj Close" ,Volume ,MACD_12_26_9 ,MACDh_12_26_9 ,MACDs_12_26_9 ,RSI_14 ,"BBL_5_2.0" ,"BBM_5_2.0" ,"BBU_5_2.0" ,"BBB_5_2.0" ,"BBP_5_2.0" ,OBV ,SMA_20 ,EMA_50 ,STOCHk_14_3_3 ,STOCHd_14_3_3 ,ADX_14 ,DMP_14 ,DMN_14 ,AD ,STDEV_30 ,VWAP_D ,PV_DERIVATIVE ,Y_CLOSE5_MIN from STOCK_DAILY WHERE ticker='TSLA'"""

print(query)

all_dataset = SqliteDataSet(query=query)

print(all_dataset.__len__())

train_set, test_set = T.utils.data.random_split(all_dataset,[650,49])

# print(all_dataset.__getitem__(0))

# test_dataset = torchvision.datasets.MNIST(root='./data',
#                                           train=False,
#                                           transform=transforms.ToTensor())
# 
# # Data loader
train_loader = T.utils.data.DataLoader(dataset=train_set,
                                           batch_size=batch_size,
                                           shuffle=True)
verify_loader = T.utils.data.DataLoader(dataset=train_set,
                                           batch_size=1,
                                           shuffle=True)
# 
test_loader = T.utils.data.DataLoader(dataset=test_set,
                                          batch_size=1,
                                          shuffle=False)

examples = iter(test_loader)
example_data, example_targets = next(examples)
example_targets, example_data

# examples = iter(train_loader)
# example_data, example_targets = next(examples)
# print(example_data)

# for i in range(6):
#     plt.subplot(2,3,i+1)
#     plt.imshow(example_data[i][0], cmap='gray')
# plt.show()

cuda
select Open ,High ,Low ,Close ,"Adj Close" ,Volume ,MACD_12_26_9 ,MACDh_12_26_9 ,MACDs_12_26_9 ,RSI_14 ,"BBL_5_2.0" ,"BBM_5_2.0" ,"BBU_5_2.0" ,"BBB_5_2.0" ,"BBP_5_2.0" ,OBV ,SMA_20 ,EMA_50 ,STOCHk_14_3_3 ,STOCHd_14_3_3 ,ADX_14 ,DMP_14 ,DMN_14 ,AD ,STDEV_30 ,VWAP_D ,PV_DERIVATIVE ,Y_CLOSE5_MIN from STOCK_DAILY WHERE ticker='TSLA'


OperationalError: (sqlite3.OperationalError) no such column: VWAP_D
[SQL: select Open ,High ,Low ,Close ,"Adj Close" ,Volume ,MACD_12_26_9 ,MACDh_12_26_9 ,MACDs_12_26_9 ,RSI_14 ,"BBL_5_2.0" ,"BBM_5_2.0" ,"BBU_5_2.0" ,"BBB_5_2.0" ,"BBP_5_2.0" ,OBV ,SMA_20 ,EMA_50 ,STOCHk_14_3_3 ,STOCHd_14_3_3 ,ADX_14 ,DMP_14 ,DMN_14 ,AD ,STDEV_30 ,VWAP_D ,PV_DERIVATIVE ,Y_CLOSE5_MIN from STOCK_DAILY WHERE ticker='TSLA']
(Background on this error at: https://sqlalche.me/e/20/e3q8)

In [11]:
import torch
##Fully connected neural network with one hidden layer
# class YhatCrossEntropyLoss(torch.CrossEntropyLoss):
#     def __init__(self)

class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNet, self).__init__()
        self.nn_block1 = nn.Sequential(
            nn.Linear(input_size, 200),
            nn.ReLU()     ,
            nn.Linear(200, 200),
            nn.ReLU(),
            nn.Linear(200, num_classes),
            # nn.Softmax(dim=1)
        )

    def forward(self, x):
        out = self.nn_block1(x)

        # no activation and no softmax at the end
        return out

model = NeuralNet(input_size, hidden_size, num_classes).to(device)

# Loss and optimizer
# weight = torch.tensor([10.,4.,1.2,1.1,0.9])
weight = torch.tensor([1.,1.,1.,1.,1.])
criterion = nn.CrossEntropyLoss(weight=weight)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Train the model
n_total_steps = len(train_loader)
for epoch in range(num_epochs):
    for i, (databatch, labels) in enumerate(train_loader):
        # origin shape: [100, 1, 28, 28]
        # resized: [100, 784]
        databatch = databatch.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        # Forward pass and loss calculation
        outputs = model(databatch)
        loss = criterion(outputs, labels)
        # for p in model.parameters():
        #     p.register_hook(lambda grad: torch.clamp(grad, -1, 1 ))
        
        # Backward and optimize
        loss.backward()
        # torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0, norm_type=2)

        optimizer.step()
       

        if (i+1) % 100 == 0:
            ##print (f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{n_total_steps}], Loss: {loss.item():.4f}')
            print("out put sample: ", outputs)
            # print(model.parameters())

out put sample:  tensor([[ 0.2274, -0.1588, -0.3679,  0.0613, -4.6641],
        [-0.3161,  0.2043,  0.5861, -1.0795, -1.8019],
        [ 0.2276, -0.1106, -0.0726, -0.1885, -1.1172]],
       grad_fn=<AddmmBackward0>)
out put sample:  tensor([[-0.3345, -1.7054,  0.9991, -0.2576, -5.0325],
        [ 0.6915, -0.5253,  0.4165, -1.3155, -2.3044],
        [-0.4189, -2.2684,  1.3101, -0.2945, -6.9314]],
       grad_fn=<AddmmBackward0>)
out put sample:  tensor([[ 0.4725,  0.2498,  0.4073, -0.3503, -1.6945],
        [ 0.3945,  0.1278,  0.2116, -0.4425, -1.0444],
        [ 0.6020,  0.0919,  0.6477, -0.7092, -2.8115]],
       grad_fn=<AddmmBackward0>)
out put sample:  tensor([[ 0.3400,  0.0745,  0.3377, -0.4756, -0.8104],
        [ 0.3400,  0.0745,  0.3377, -0.4756, -0.8104],
        [ 0.4748, -0.2900,  0.7620,  0.2866, -2.5295]],
       grad_fn=<AddmmBackward0>)
out put sample:  tensor([[ 0.5347,  0.1371,  0.3246, -0.6924, -1.0579],
        [ 0.9639,  0.5974,  0.5713, -1.8596, -3.1302],
        [

In [13]:
from pathlib import Path
torch.save(model.state_dict(), "./tamed/M1_min5_46v_20240106.pt")

In [14]:
# Test the model: we don't need to compute gradients
import torch as T
with T.no_grad():
    n_correct = 0
    n_samples = len(test_loader.dataset)
    print(n_samples)

    for d_data, labels in test_loader:

        labels = labels.to(device)
        outputs = model(d_data)

        # max returns (output_value ,index)
        _, predicted = T.max(outputs, 1)
        print(labels,predicted)
        n_correct += (predicted == labels).sum().item()

    acc = n_correct / n_samples
    print(f'Accuracy of the network on the {n_samples} test images: {100*acc} %')

49
tensor([1]) tensor([0])
tensor([2]) tensor([0])
tensor([2]) tensor([2])
tensor([0]) tensor([0])
tensor([3]) tensor([0])
tensor([0]) tensor([0])
tensor([2]) tensor([0])
tensor([3]) tensor([0])
tensor([2]) tensor([0])
tensor([0]) tensor([0])
tensor([2]) tensor([0])
tensor([4]) tensor([0])
tensor([2]) tensor([0])
tensor([0]) tensor([0])
tensor([0]) tensor([0])
tensor([2]) tensor([0])
tensor([2]) tensor([2])
tensor([2]) tensor([0])
tensor([2]) tensor([0])
tensor([0]) tensor([0])
tensor([3]) tensor([0])
tensor([1]) tensor([0])
tensor([2]) tensor([0])
tensor([0]) tensor([0])
tensor([0]) tensor([0])
tensor([1]) tensor([0])
tensor([0]) tensor([0])
tensor([2]) tensor([0])
tensor([3]) tensor([0])
tensor([0]) tensor([0])
tensor([3]) tensor([2])
tensor([3]) tensor([0])
tensor([1]) tensor([2])
tensor([2]) tensor([0])
tensor([0]) tensor([0])
tensor([2]) tensor([0])
tensor([0]) tensor([0])
tensor([2]) tensor([2])
tensor([1]) tensor([0])
tensor([2]) tensor([0])
tensor([1]) tensor([0])
tensor([3]) t

In [17]:
print(model.parameters())

<generator object Module.parameters at 0x0000012D74DEA580>


In [16]:
l_model = NeuralNet(input_size, hidden_size, num_classes)
l_model.load_state_dict(torch.load('./tamed/M1_min5_46v_20240106.pt'))


<All keys matched successfully>

In [17]:
l_model.eval()
# Test the model: we don't need to compute gradients
import torch as T
with T.no_grad():
    n_correct = 0
    n_samples = len(test_loader.dataset)
    print(n_samples)

    for d_data, labels in test_loader:

        labels = labels.to(device)
        outputs = l_model(d_data)

        # max returns (output_value ,index)
        _, predicted = T.max(outputs, 1)
        print(labels,predicted)
        n_correct += (predicted == labels).sum().item()

    acc = n_correct / n_samples
    print(f'Accuracy of the network on the {n_samples} test images: {100*acc} %')


49
tensor([1]) tensor([0])
tensor([2]) tensor([0])
tensor([2]) tensor([2])
tensor([0]) tensor([0])
tensor([3]) tensor([0])
tensor([0]) tensor([0])
tensor([2]) tensor([0])
tensor([3]) tensor([0])
tensor([2]) tensor([0])
tensor([0]) tensor([0])
tensor([2]) tensor([0])
tensor([4]) tensor([0])
tensor([2]) tensor([0])
tensor([0]) tensor([0])
tensor([0]) tensor([0])
tensor([2]) tensor([0])
tensor([2]) tensor([2])
tensor([2]) tensor([0])
tensor([2]) tensor([0])
tensor([0]) tensor([0])
tensor([3]) tensor([0])
tensor([1]) tensor([0])
tensor([2]) tensor([0])
tensor([0]) tensor([0])
tensor([0]) tensor([0])
tensor([1]) tensor([0])
tensor([0]) tensor([0])
tensor([2]) tensor([0])
tensor([3]) tensor([0])
tensor([0]) tensor([0])
tensor([3]) tensor([2])
tensor([3]) tensor([0])
tensor([1]) tensor([2])
tensor([2]) tensor([0])
tensor([0]) tensor([0])
tensor([2]) tensor([0])
tensor([0]) tensor([0])
tensor([2]) tensor([2])
tensor([1]) tensor([0])
tensor([2]) tensor([0])
tensor([1]) tensor([0])
tensor([3]) t